____
**Universidad Tecnológica Nacional, Buenos Aires**

**Ingeniería Industrial**

**Cátedra de Ciencia de Datos - Turno jueves noche**

**Elaborado por: Mareque Lucas**

**Editado por: Aguirre Nicolas**
____

# Librerias

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC

# Dataset

El objetivo es tratar de predecir  el tipo de medicamento que podría ser adecuado para un paciente.

Las características que se utilizarán como base para la predicción incluyen:

- **Edad**
- **Sexo**
- **Niveles de presión arterial (BP)**
- **Niveles de colesterol**
- **Relación sodio/potasio**

Los tipos de medicamentos analizados son **Y, C, X, A o B**. Dado que existen cinco clases posibles, se trata de un problema de clasificación multiclase.

[Fuente del dataset](https://www.kaggle.com/datasets/prathamtripathi/drug-classification/data)

In [ ]:
pwd

In [ ]:
csv_path = "path/to/dataset_medicamentos.csv"
df = pd.read_csv(csv_path)

In [ ]:
df.head()

# EDA

**Tarea:**

- Imprimir la matriz de correlación

- Histograma para Na_to_K

- Boxplot para ver Edad en función de la clase (BP).

# Preprocessing

In [ ]:
# Verificamos si hay valores no validos
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

In [ ]:
# Son pocas las filas con NaN, asi que las eliminamos
df.dropna(inplace=True)

In [ ]:
# Información del dataframe
df.info()

En la columna Dtype hay varios tipos de datos diferentes: int64, float64 y object.
Los datos con tipo object son datos categóricos, por lo que después se debe realizar un preprocesamiento para convertirlos en datos numéricos.

In [ ]:
# Separamos las features del target

features = ['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K',]
target = ['Drug']


X = df.loc[:,features]
Y = df.loc[:,target]

In [ ]:
Y

- **Para convertir las features categoricas en datos numéricos**, podemos utilizar get_dummies de la librería Pandas.

In [ ]:
categorical_data = ['Sex','BP','Cholesterol']
X = pd.get_dummies(X, columns=categorical_data)
X.head()

Ahora se puede observar que las variables categoricas son reemplazadas por columnas por cada clase. Por ejemplo, en la columna BP hay tres clases: BP_HIGH,	BP_LOW y BP_NORMAL.


- **Para convertir el target en datos numéricos**, podemos utilizar LabelEncoder de sklearn.

In [ ]:
# Definimos la instancia del objeto LabelEncoder
le = LabelEncoder()
# Ajustamos y transformamos la variable objetivo
Y_encoded = le.fit_transform(Y)

print("Clases:", le.classes_)

In [ ]:
df_temp = pd.DataFrame({
    "Drug_original": Y.values.ravel(),
    "Label": Y_encoded
})

df_temp

# Train & Evaluate

In [ ]:
# Separamos train y test

x_train, x_test, y_train, y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=34)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(f"Los datos se dividieron en cuatro partes: {x_train.shape[0]} datos para entrenamiento y {x_test.shape[0]} datos para test")


- **Utilizamos StandardScaler** para la variable numérica Na_to_K

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train[["Na_to_K"]])

x_train["Na_to_K"] = scaler.transform(x_train[["Na_to_K"]])
x_test["Na_to_K"] = scaler.transform(x_test[["Na_to_K"]])

Los datos de testeo no se utilizan para el entrenamiento, unicamente se transforman. Sino le estamos pasando información que influyen en el resultado del perfomance del modelo ==> **Data leakage** 

## SVC 

In [ ]:
# Creamos una instancia de la clase SVC con el nombre model.
model = SVC()

model.fit(x_train,y_train)  # entrenamos con los datos que separamos
prediction = model.predict(x_test) # obtenemos las predicciones y las guardamos en una variable

In [ ]:
prediction

In [ ]:
# Evaluamos el modelo
model.score(x_test,y_test)

In [ ]:
# Veamos la matriz de confusion ...
cm1 = confusion_matrix(y_test, prediction)
# Visualizamos la matriz de confusión
plt.figure(figsize=(10,7))
sns.heatmap(cm1, annot=True)
plt.xlabel('Predicted Value')
plt.ylabel('True Value')

## Discusion: 
**Que se observa?**

## Grid search 

In [ ]:
# Creemos un dict con los hyperparametros que queremos optimizar de un SVC.
grid = {
    'C':[0.001, 0.01,0.1,1,10],
    'kernel' : ["linear","poly","rbf","sigmoid"],
    'degree' : [1,3,5,7],
    'gamma' : [0.001, 0.01, 1]
}

# Creamos la instancia del modelo
svm  = SVC ()
# Creamos la instancia del GridSearchCV a la que le pasamos el modelo, la grilla de hyperparametros y la cantidad de folds para cross validation.
svm_cv = GridSearchCV(
    estimator= svm, # El modelo que queremos optimizar
    param_grid= grid, # La grilla de hyperparametros donde vamos a buscar
    cv = 5, # Numero de folds para cross validation
    refit=True, # Volver a entrenar el modelo con los mejores parametros
    verbose=1
    )

In [ ]:
svm_cv.fit(x_train,y_train)

In [ ]:
# Vemos los mejores parametros encontrados y los scores
print("Best Parameters:",svm_cv.best_params_)
print("Train Score:",svm_cv.best_score_)
print("Test Score:",svm_cv.score(x_test,y_test))

El usar refit=True en GridSearchCV nos evita tener luego que definir nuevamente el modelo con los mejores hiperparametros,

```python
model_opt = SVC(
    C=svm_cv.best_params_['C'], 
    degree=svm_cv.best_params_['degree'], 
    gamma=svm_cv.best_params_['gamma'], 
    kernel=svm_cv.best_params_['kernel']
)
model_opt.fit(x_train, y_train)  # entrenamos con los datos que separamos
prediction = model_opt.predict(x_test) # obtenemos las predicciones y las guardamos en una variable
```

In [ ]:
# Al haber utilizado svm_cv con refit=True, el estimador ya se enbtreno una ultima vez
# con los mejores parametros encontrados, pero ahora agregando el fold que faltaba.
prediction = svm_cv.predict(x_test) # obtenemos las predicciones y las guardamos en una variable
prediction

In [ ]:
cm1 = confusion_matrix(y_test, prediction)
plt.figure(figsize=(10,7))
sns.heatmap(cm1,annot=True)
plt.xlabel('Predicted Value')
plt.ylabel('True Value')

# Preguntas?